# Extração e agrupamento de labels

## Conexão com o Drive para acessar os dados

Esta etapa é necessária para permitir que o Colab acesse os arquivos armazenados no Google Drive.


In [ ]:
# Conexão com o drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importação das bibliotecas

Importaremos as bibliotecas necessárias para o processamento das imagens e manipulação dos dados:

- **os**: Para interagir com o sistema operacional, listando arquivos e manipulando caminhos.
- **cv2 (OpenCV)**: Utilizada para leitura, escrita e manipulação de imagens.
- **numpy (np)**: Biblioteca para manipulação eficiente de arrays e operações matemáticas.
- **h5py**: Para salvar e carregar arquivos no formato HDF5, ideal para grandes volumes de dados.
- **google.colab.drive**: Permite montar o Google Drive no Colab para acessar arquivos armazenados na nuvem.

In [ ]:
import os
import h5py
import cv2
import numpy as np

## Extração dos dados



### Caminho para as pastas

Este trecho de código define um dicionário `paths` que contém os caminhos para as pastas de imagens X e Y de cada grupo (G1, G2, G3, G4) organizados por data.

In [ ]:
paths = {
    'G1': {
        '01-08-2024': {
            'X': '/content/drive/MyDrive/CattleImageRepository/G1/01-08-2024/X',
            'Y': '/content/drive/MyDrive/CattleImageRepository/G1/01-08-2024/y'
        },
        '08-08-2024': {
            'X': '/content/drive/MyDrive/CattleImageRepository/G1/08-08-2024/X',
            'Y': '/content/drive/MyDrive/CattleImageRepository/G1/08-08-2024/Y'
        }
    },
    'G2': {
        '06-08-2024_Video_1': {
            'X': '/content/drive/MyDrive/CattleImageRepository/G2/06-08/Video_1/X',
            'Y': '/content/drive/MyDrive/CattleImageRepository/G2/06-08/Video_1/Y'
        },
        '29-07-2024': {
            'X': '/content/drive/MyDrive/CattleImageRepository/G2/29-07/00000000196000400/X',
            'Y': '/content/drive/MyDrive/CattleImageRepository/G2/29-07/00000000196000400/Y'
        }
    },
    'G3': {
        '30-07-2024': {
            'X': '/content/drive/MyDrive/CattleImageRepository/G3/30-07/termica/x',
            'Y': '/content/drive/MyDrive/CattleImageRepository/G3/30-07/termica/y'
        }
    },
    'G4': {
        '02-08-2024': {
            'X': '/content/drive/MyDrive/CattleImageRepository/G4/02-08/X',
            'Y': '/content/drive/MyDrive/CattleImageRepository/G4/02-08/Y'
        }
    },
}

### Função para processar as imagens dos grupos, sem condições

Essa função serve para carregar as imagens das pastas de cada grupo, sem aplicar nenhuma regra específica para considerar os arquivos, a fim de analisar o shape das pastas correspondentes de X e Y, verificando se possuem a mesma quantidade de arquivos ou se está corrompida de alguma forma.

In [ ]:
def load_images_from_folder(folder, target_size=(128, 128)):
  images = []
  for filename in os.listdir(folder):

    img = cv2.imread(os.path.join(folder, filename))
    if img is not None:
        if target_size:
            img = cv2.resize(img, target_size)
        images.append(img)
  return np.array(images)

#### Carregamento das imagens dos grupos

O código a seguir carrega as imagens de todas as pastas que estão destacadas na variável `paths`.

In [ ]:
group_data = {}

for group, dates in paths.items():
    group_data[group] = {}
    for date, folders in dates.items():
        X_path = folders['X']
        Y_path = folders['Y']

        X_images = load_images_from_folder(X_path)
        Y_images = load_images_from_folder(Y_path)

        group_data[group][date] = {
            'X': X_images,
            'Y': Y_images
        }


#### Verificação dos shapes das pastas sem condições

In [ ]:
for group, data in group_data.items():
    for date, images in data.items():
        print(f"Grupo: {group}, Data: {date}, X shape: {images['X'].shape}, Y shape: {images['Y'].shape}")

Grupo: G1, Data: 01-08-2024, X shape: (763, 224, 224, 3), Y shape: (745, 224, 224, 3)
Grupo: G1, Data: 08-08-2024, X shape: (615, 224, 224, 3), Y shape: (829, 224, 224, 3)
Grupo: G2, Data: 06-08-2024_Video_1, X shape: (856, 224, 224, 3), Y shape: (856, 224, 224, 3)
Grupo: G2, Data: 29-07-2024, X shape: (325, 224, 224, 3), Y shape: (342, 224, 224, 3)
Grupo: G3, Data: 30-07-2024, X shape: (724, 224, 224, 3), Y shape: (724, 224, 224, 3)
Grupo: G4, Data: 02-08-2024, X shape: (843, 224, 224, 3), Y shape: (843, 224, 224, 3)


É possível verificar claramente que há distorção nas pastas do G1 e do G2, em quesitos de quantidade de arquivos nas pastas X e Y. Portanto, precisamos aplicar condições para assegurar que apenas os arquivos X e Y correspondentes sejam considerados.

### Funções para processar os arquivos de cada grupo, com condições

Essas funções servem para carregar as imagens das pastas de cada grupo, aplicando as regras específicas para garantir que apenas os arquivos correspondentes de X e Y sejam considerados. As funções estão separadas por grupo devido às diferenças de nomenclatura e organização de arquivos.

In [ ]:
# Processamento para cada grupo
X_data, Y_data = [], []

#### Função para processar arquivos do G1

Esta função carrega os arquivos X e Y do Grupo 1, verificando as variações de nomenclatura nos arquivos Y (`_x_y` ou `_y`).

In [ ]:
def process_g1(x_dir, y_dir):
    X_data, Y_data = [], []
    x_files = [f for f in os.listdir(x_dir) if not 'overlay' in f]
    y_files = [f for f in os.listdir(y_dir) if not 'overlay' in f]

    for x_file in x_files:
        base_name = x_file.rsplit('_x', 1)[0]
        y_file_1 = base_name + '_x_y.png'
        y_file_2 = base_name + '_y.png'

        if y_file_1 in y_files:
            y_file = y_file_1
        elif y_file_2 in y_files:
            y_file = y_file_2
        else:
            print(f"Corresponding Y file not found for {x_file}. Skipping...")
            continue

        x_img = cv2.imread(os.path.join(x_dir, x_file))
        y_img = cv2.imread(os.path.join(y_dir, y_file))
        X_data.append(x_img)
        Y_data.append(y_img)

    print(f"Total processed for group G1: {len(X_data)} pairs.")
    return np.array(X_data), np.array(Y_data)


Nesta célula, processamos as imagens do Grupo 1. Para cada data no Grupo 1, a função processa as imagens de X e Y, garantindo que haja correspondência entre elas. Os dados correspondentes são armazenados em arrays numpy. Após o processamento, os dados do Grupo 1 são salvos em um arquivo HDF5.


In [ ]:
# Processando G1
for date in paths['G1']:
    X_g1, Y_g1 = process_g1(paths['G1'][date]['X'], paths['G1'][date]['Y'])
    print(f"G1, Data: {date}, X shape: {X_g1.shape}, Y shape: {Y_g1.shape}")
    X_data.append(X_g1)
    Y_data.append(Y_g1)

# Salvando os dados processados do Grupo 1 em um arquivo HDF5
with h5py.File('/content/drive/MyDrive/data/G1_data.h5', 'w') as hf:
    hf.create_dataset('X', data=np.concatenate(X_data))
    hf.create_dataset('Y', data=np.concatenate(Y_data))

Corresponding Y file not found for 01-08-24_video1_00000000205000400_107880_1_x.png. Skipping...
Total processed for group G1: 744 pairs.
G1, Data: 01-08-2024, X shape: (744, 128, 128, 3), Y shape: (744, 128, 128, 3)
Total processed for group G1: 613 pairs.
G1, Data: 08-08-2024, X shape: (613, 128, 128, 3), Y shape: (613, 128, 128, 3)


#### Função para processar arquivos do G2
Esta função é para o Grupo 2, onde é verificado se o arquivo de imagem X termina com `_x1`, `_x2`, `_x3`, ou `_x4`, e a correspondência é feita com o arquivo Y. Além disso, é preciso fazer o resize das imagens para poder fazer o append.


In [ ]:
def process_g2(x_dir, y_dir):
    X_data, Y_data = [], []
    x_files = [f for f in os.listdir(x_dir) if not 'overlay' in f and ('_x1' in f or '_x2' in f or '_x3' in f or '_x4' in f)]
    y_files = [f for f in os.listdir(y_dir) if not 'overlay' in f]

    for x_file in x_files:
        # Substitui o sufixo '_x' por '_y' para encontrar o arquivo Y correspondente
        base_name = x_file.replace('_x', '_y')
        if base_name in y_files:
            y_file = base_name
            x_img = cv2.imread(os.path.join(x_dir, x_file))
            y_img = cv2.imread(os.path.join(y_dir, y_file))

            # Fazendo o resize
            target_size=(128, 128)
            x_img = cv2.resize(x_img, target_size)
            y_img = cv2.resize(y_img, target_size)

            X_data.append(x_img)
            Y_data.append(y_img)
        else:
            print(f"Corresponding Y file not found for {x_file}. Skipping...")

    print(f"Total processed for group G2: {len(X_data)} pairs.")
    return np.array(X_data), np.array(Y_data)


Nesta célula, processamos as imagens do Grupo 2. O processamento aqui verifica arquivos X com sufixos específicos ('x1', 'x2', etc.) e encontra correspondência com Y. Após o processamento, os dados do Grupo 2 são salvos em um arquivo HDF5.


In [ ]:
# Processando G2
for date in paths['G2']:
    X_g2, Y_g2 = process_g2(paths['G2'][date]['X'], paths['G2'][date]['Y'])
    print(f"G2, Data: {date}, X shape: {X_g2.shape}, Y shape: {Y_g2.shape}")
    X_data.append(X_g2)
    Y_data.append(Y_g2)


# Salvando os dados processados do Grupo 2 em um arquivo HDF5
with h5py.File('/content/drive/MyDrive/data/G2_data.h5', 'w') as hf:
    hf.create_dataset('X', data=np.concatenate(X_data))
    hf.create_dataset('Y', data=np.concatenate(Y_data))

Total processed for group G2: 515 pairs.
G2, Data: 06-08-2024_Video_1, X shape: (515, 128, 128, 3), Y shape: (515, 128, 128, 3)
Total processed for group G2: 260 pairs.
G2, Data: 29-07-2024, X shape: (260, 128, 128, 3), Y shape: (260, 128, 128, 3)


#### Função para processar arquivos do G3
No Grupo 3, a correspondência é direta entre arquivos de X e Y, onde a única diferença é a substituição de `_x` por `_y`.


In [ ]:
def process_g3(x_dir, y_dir):
    X_data, Y_data = [], []
    x_files = [f for f in os.listdir(x_dir) if not 'overlay' in f]
    y_files = [f for f in os.listdir(y_dir) if not 'overlay' in f]


    for x_file in x_files:
        base_name = x_file.replace('_x', '_y')

        # Imprimir o nome base e o nome esperado para Y
        print(f"Procurando o arquivo Y correspondente para {x_file}")
        print(f"Expected Y file: {base_name}")

        if base_name in y_files:
            print(f"Encontrou o arquivo Y correspondente: {base_name}")
            x_img = cv2.imread(os.path.join(x_dir, x_file))
            y_img = cv2.imread(os.path.join(y_dir, base_name))
            X_data.append(x_img)
            Y_data.append(y_img)
        else:
            print(f"Corresponding Y file not found for {x_file}. Skipping...")

    print(f"Total processed for group G3: {len(X_data)} pairs.")
    return np.array(X_data), np.array(Y_data)


Nesta célula, processamos as imagens do Grupo 3. A correspondência é direta entre os arquivos X e Y, com apenas uma substituição de sufixo. Após o processamento, os dados do Grupo 3 são salvos em um arquivo HDF5.


In [ ]:
# Processando G3
for date in paths['G3']:
    X_g3, Y_g3 = process_g3(paths['G3'][date]['X'], paths['G3'][date]['Y'])
    print(f"G3, Data: {date}, X shape: {X_g3.shape}, Y shape: {Y_g3.shape}")
    X_data.append(X_g3)
    Y_data.append(Y_g3)

# Salvando os dados processados do Grupo 3 em um arquivo HDF5
with h5py.File('/content/drive/MyDrive/data/G3_data.h5', 'w') as hf:
    hf.create_dataset('X', data=np.concatenate(X_data))
    hf.create_dataset('Y', data=np.concatenate(Y_data))

Procurando o arquivo Y correspondente para 30-07-24_video_00000000199000400_frame_0008_x2.png
Expected Y file: 30-07-24_video_00000000199000400_frame_0008_y2.png
Encontrou o arquivo Y correspondente: 30-07-24_video_00000000199000400_frame_0008_y2.png
Procurando o arquivo Y correspondente para 30-07-24_video_00000000199000400_frame_0077_x2.png
Expected Y file: 30-07-24_video_00000000199000400_frame_0077_y2.png
Encontrou o arquivo Y correspondente: 30-07-24_video_00000000199000400_frame_0077_y2.png
Procurando o arquivo Y correspondente para 30-07-24_video_00000000199000400_frame_0073_x1.png
Expected Y file: 30-07-24_video_00000000199000400_frame_0073_y1.png
Encontrou o arquivo Y correspondente: 30-07-24_video_00000000199000400_frame_0073_y1.png
Procurando o arquivo Y correspondente para 30-07-24_video_00000000199000400_frame_0031_x1.png
Expected Y file: 30-07-24_video_00000000199000400_frame_0031_y1.png
Encontrou o arquivo Y correspondente: 30-07-24_video_00000000199000400_frame_0031_y1.

#### Função para processar arquivos do G4
Semelhante ao Grupo 3, mas específico para a nomenclatura do Grupo 4.


In [ ]:
def process_g4(x_dir, y_dir):
    X_data, Y_data = [], []
    x_files = [f for f in os.listdir(x_dir) if not 'overlay' in f]
    y_files = [f for f in os.listdir(y_dir) if not 'overlay' in f]

    for x_file in x_files:
        y_file = x_file.replace('_x', '_y')

        if y_file in y_files:
            x_img = cv2.imread(os.path.join(x_dir, x_file))
            y_img = cv2.imread(os.path.join(y_dir, y_file))
            X_data.append(x_img)
            Y_data.append(y_img)

    return np.array(X_data), np.array(Y_data)

Nesta célula, processamos as imagens do Grupo 4. A correspondência entre X e Y é muito clara, com substituição simples do sufixo. Após o processamento, os dados do Grupo 4 são salvos em um arquivo HDF5.


In [ ]:
# Processando G4
X_g4, Y_g4 = process_g4(paths['G4']['02-08-2024']['X'], paths['G4']['02-08-2024']['Y'])
print(f"G4, Data: 02-08-2024, X shape: {X_g4.shape}, Y shape: {Y_g4.shape}")
X_data.append(X_g4)
Y_data.append(Y_g4)

# Salvando os dados processados do Grupo 4 em um arquivo HDF5
with h5py.File('/content/drive/MyDrive/data/G4_data.h5', 'w') as hf:
    hf.create_dataset('X', data=np.concatenate(X_data))
    hf.create_dataset('Y', data=np.concatenate(Y_data))

G4, Data: 02-08-2024, X shape: (843, 128, 128, 3), Y shape: (843, 128, 128, 3)


#### Verificação dos shapes das pastas com condições
Verificamos as dimensões das imagens X e Y em cada uma das pastas para garantir que elas tenham as mesmas dimensões. Esse passo é importante para garantir a compatibilidade durante o treinamento do modelo.

### Salvando em um conjunto de dados
Aqui, os dados processados e verificados são salvos em um arquivo HDF5, que é um formato de armazenamento eficiente, especialmente útil para grandes volumes de dados, permitindo carregamento rápido para futuras análises.


In [ ]:
# Salvando o conjunto de dados combinado de todos os grupos
with h5py.File('/content/drive/MyDrive/data/combined_data.h5', 'w') as hf:
    hf.create_dataset('X', data=np.concatenate([X_g1, X_g2, X_g3, X_g4]))
    hf.create_dataset('Y', data=np.concatenate([Y_g1, Y_g2, Y_g3, Y_g4]))

